In [1]:
from PIL import Image
import cv2
import pytesseract
import fitz
import pandas as pd
import numpy as np
import os
import re
import nltk
from pytesseract import Output

#path to Tesseract
path_to_tesseract = r"C:\Program Files\Tesseract-OCR\Tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = path_to_tesseract

In [2]:
import re
# function for splitting sentences


def split_into_sentences(text):
    alphabets= "([A-Za-z])"
    prefixes = "(Mr|St|Mrs|Ms|Dr|No)[.]"
    suffixes = "(Inc|Ltd|Jr|Sr|Co|No)"
    starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
    acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
    websites = "[.](com|net|org|io|gov|ru|su|www)"
    digits = "([0-9])"
    
    
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(" " + digits + "[.]" + digits," \\1<prd>\\2",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences



In [3]:
#функция для очистки текста от лишних символов
def cleantext(row):
    t = row['text']
    try:
        clean_text =  " ".join((re.sub(r'[^a-zA-Z ]', ' ', t)).split())
        if len(clean_text)<4: clean_text = 'NA' #чтобы избавиться от коротких предложений
        return clean_text
    except:
        print('cleantext - очистка текста, проблема с входными данными')
        

In [4]:
%%time
#чтение файлов и загрузка данных в датафрейм df_test
zoom = 2
mat = fitz.Matrix(zoom, zoom)


df_test = pd.DataFrame(data=None, columns =['file','page','left','top','width','height','text'])
path = './pdfs/' #путь к папке с файлами для чтения
i = 0
line=0

for element in os.listdir(path):
    if element[-3:] == 'pdf':
        doc = fitz.open(path+ '/' + element)
        print('document:',element,'pages:',doc.pageCount)
        #for current_page in range(91,92): #для отладки
        for current_page in range(0,doc.pageCount):
            print('precessing page',current_page + 1, 'of',doc.pageCount) #+1 чтобы не с 0 страницы
            pagetext = ''
            page = doc.loadPage(current_page)
            pix = page.getPixmap(matrix = mat)
            pix.writePNG('new_file.png')
            image = cv2.imread('new_file.png')
            box_df = pytesseract.image_to_data(image, output_type=Output.DATAFRAME)
            
            df_coordinates = pd.DataFrame(data=None,columns = ['block','par_num','l','t','w','h']) #вычисляем координаты pap_num'ов и пишем в df_coordinates
            for block in box_df['block_num'].unique(): 
                tempdf = box_df.query('block_num == @block')
                for pn in tempdf['par_num'].unique():
                    tempdf2 = tempdf.query('par_num == @pn')
                    min_left = tempdf2['left'].min()
                    min_top = tempdf2['top'].min()
                    max_width = tempdf2['width'].max()
                    max_height = tempdf2['height'].max()
                    df_coordinates.loc[line] = (block,pn,min_left,min_top,max_width,max_height)
                    line+=1
                    
            
            box_df.dropna(subset=['text'],inplace = True) #удаление пустых строк, после формирования блоков - иначе границы неправильные
            
            df_text_blocks = pd.DataFrame(data=None, columns = ['file','page','left','top','width','height','text']) #новый датафрейм по блокам
            display(df_coordinates)
            p=0 #счетчик для par_mun, далее перебираем
            for block in box_df['block_num'].unique(): #создаем новый датафрейм
                tempdf = box_df.query('block_num == @block') #делаем срез по номерам блоков
                for pn in tempdf['par_num'].unique():
                    tempdf2 = tempdf.query('par_num == @pn') #делаем срез по par_num
                    string_text = ' '.join(w for w in tempdf2['text']) # соединяем все слова par_num в строку
                    if string_text[-1] != '.' : string_text += '.' #в конце строки добавим точку там, где её не было
                    if len(string_text) <= 4: continue #короткие строки не нужны

                    coords = df_coordinates.query('(block == @block)and(par_num == @pn)').copy()
                    coords = coords.reset_index(drop = True)
                    df_text_blocks.loc[p] = (element, current_page+1, #+1 чтобы страницы были не с нуля, сохраняем координаты блока
                                             coords['l'][0],coords['t'][0],
                                             coords['w'][0],coords['h'][0],
                                             string_text)
                    p += 1
                                         
                
            df_test = df_test.append(df_text_blocks)
            
            i += 1

document: ALROSA-Annual-Report-2019.pdf pages: 140
precessing page 92 of 140


,block,par_num,l,t,w,h
0,0,0,0,0,2382,1684
1,1,0,496,65,198,9
2,1,1,496,65,198,9
3,2,0,87,213,541,18
4,2,1,87,213,541,18
...,...,...,...,...,...,...
64,30,1,1920,1024,346,2
65,31,0,1429,1443,837,4
66,31,1,1429,1443,837,4
67,32,0,1276,1570,194,12


Wall time: 3.67 s


In [5]:
display(box_df)
encd = 'cp1252' #кодировка для выгружаемого файла, для русского языка 'uft-8-sig'
box_df.to_excel('box_df.xls', index = False, encoding = encd)


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
4,5,1,1,1,1,1,496,65,51,9,90,ALROSA
5,5,1,1,1,1,2,552,65,53,9,96,ANNUAL
6,5,1,1,1,1,3,611,65,49,9,96,REPORT
7,5,1,1,1,1,4,665,65,29,9,96,2019
11,5,1,2,1,1,1,87,213,197,18,96,INFORMATION
...,...,...,...,...,...,...,...,...,...,...,...,...
588,5,1,32,1,1,2,1316,1571,38,10,91,224-FZ
589,5,1,32,1,1,3,1360,1571,29,10,96,dated
590,5,1,32,1,1,4,1395,1571,21,10,96,July
591,5,1,32,1,1,5,1421,1571,16,11,80,"27,"


In [6]:
#выгрузка DF в csv файлы
encd = 'cp1252' #кодировка для выгружаемого файла, для русского языка 'uft-8-sig'
df_test.to_excel('output.xls', index = False, encoding = encd)
print(display(df_test))

,file,page,left,top,width,height,text
0,ALROSA-Annual-Report-2019.pdf,92,496,65,198,9,ALROSA ANNUAL REPORT 2019.
1,ALROSA-Annual-Report-2019.pdf,92,87,213,541,18,INFORMATION DISCLOSURE CHANNELS.
2,ALROSA-Annual-Report-2019.pdf,92,86,271,480,160,One of the key communication channels is the C...
3,ALROSA-Annual-Report-2019.pdf,92,614,281,483,58,"In addition to the corporate site, the Company..."
4,ALROSA-Annual-Report-2019.pdf,92,97,541,936,87,Disclosure channel Info/audience Corporate sit...
5,ALROSA-Annual-Report-2019.pdf,92,98,633,808,39,http://wwwaalrosa.ru — information about mater...
6,ALROSA-Annual-Report-2019.pdf,92,513,699,515,35,— information about securities issued by the C...
7,ALROSA-Annual-Report-2019.pdf,92,513,765,545,17,"— quarterly, annual and CSR reports (sustainab..."
8,ALROSA-Annual-Report-2019.pdf,92,513,809,318,13,— annual and interim financial statements.
9,ALROSA-Annual-Report-2019.pdf,92,513,853,581,39,— materials to the General Meetings of Shareho...


None


In [7]:
#разделение текстов страниц на предложения и предобработка

df_test['text'] = df_test['text'].apply(split_into_sentences) #применяем функцию разделения на предложения
df_test = df_test.explode('text').reset_index(drop = True) #разделение строк на предложения

df_test.dropna(subset = ['text'], inplace = True) # удаление пустых строк

df_test = df_test.drop(df_test[ df_test['text'] == '.'].index) # удаляем предложения состоящие из одной точки
df_test = df_test.reset_index(drop = True) #обновляем индекс
print(display(df_test))

encd = 'cp1252' #кодировка для выгружаемого файла, для русского языка 'uft-8-sig'
df_test.to_excel('df_test_ready.xls', index = False, encoding = encd)

,file,page,left,top,width,height,text
0,ALROSA-Annual-Report-2019.pdf,92,496,65,198,9,ALROSA ANNUAL REPORT 2019.
1,ALROSA-Annual-Report-2019.pdf,92,87,213,541,18,INFORMATION DISCLOSURE CHANNELS.
2,ALROSA-Annual-Report-2019.pdf,92,86,271,480,160,One of the key communication channels is the C...
3,ALROSA-Annual-Report-2019.pdf,92,86,271,480,160,"alrosa.ru, containing not only information and..."
4,ALROSA-Annual-Report-2019.pdf,92,86,271,480,160,"Concurrently, information is posted on the Eng..."
5,ALROSA-Annual-Report-2019.pdf,92,86,271,480,160,alrosa.ru.
6,ALROSA-Annual-Report-2019.pdf,92,614,281,483,58,"In addition to the corporate site, the Company..."
7,ALROSA-Annual-Report-2019.pdf,92,97,541,936,87,Disclosure channel Info/audience Corporate sit...
8,ALROSA-Annual-Report-2019.pdf,92,98,633,808,39,http://wwwaalrosa.ru — information about mater...
9,ALROSA-Annual-Report-2019.pdf,92,513,699,515,35,— information about securities issued by the C...


None


In [8]:
############################
#RECTANGLE DRAWING FUNCTION#
############################
# "import fitz" before function use
#INPUT:
# - DataFrame must contain columns: 'block_num','file','page','left','top','width','height';
#OUTPUT:
# - pictures
############################
#  settings:
# color - RGB, (0,255,0) == green
# path - path to file-containing folder (like './pdf_files)
# thickness - box line thickness in pixels
# indention - indention from box coordinates (bow will be bigger)
# zoom - fitz.Matrix(zoom, zoom) which used to read pictures previously
# 

def draw_rectangle(df):
    
    ####settings: #####
    path = './pdfs'
    color = (0,255,0)
    thickness = 2
    indention = 2
    zoom = 2
    ###################
    
    try:
        mat = fitz.Matrix(zoom, zoom)
        df_temp = df.query('left > @indention') #чтобы случайно не вылезти за картинку влево
        for file in df_temp['file'].unique(): # отдельно по каждому файлу
            for page in df_temp.query('file == @file')['page'].unique(): #отдельно по каждой странице в файле
                df_page_temp = df_temp.query('(page == @page) & (file == @file)')
                df_page_temp.reset_index(drop = True)
                doc = fitz.open(path + '/' + file)
                pagenum = doc.loadPage(int(page) - 1) #ранее начинали с первой страницы для анализа, технически начало с нулевой
                pix = pagenum.getPixmap(matrix=mat) #mat-матрица увеличения, задается при загрузке библиотек, требуется если читали картинку с увеличением
                pix.writePNG('./temp/new_file.png')
                image = cv2.imread('./temp/new_file.png')
                for ind in list(df_page_temp.index): # рисуем все блоки на странице, ничего страшного если два раза отрисуем один и тот же
                    (x,y,w,h) = (df_page_temp['left'][ind],df_page_temp['top'][ind],df_page_temp['width'][ind],df_page_temp['height'][ind])
                    cv2.rectangle(image,(x-indention,y-indention),(x+w+int(2*indention),y+h+int(2*indention)),color,thickness)
                page_name = str(path) + '/' + str(file[:-4])  +'_page_' + str(page) + '.png' #формируем путь и имя картинки
                print(page_name)
                cv2.imwrite(page_name,image) #записываем на диск
        return
    except:
        print('проблема с входными данными, возможно не указан файл или он отсутствует по пути, указанном в settings')
        
                 
                
            




In [9]:
draw_rectangle(df_test)

./pdfs/ALROSA-Annual-Report-2019_page_92.png
